In [3]:
from datasets import load_dataset
import random
import pandas as pd

# Define dataset names and load them from Hugging Face
dataset_names = [
    'fmars/wiki_stem',
    'deepmind/aqua_rat',
    'HuggingFaceTB/openstax_paragraphs',
    'openlifescienceai/medmcqa',
    'allenai/qasc'
]

# Dictionary to hold loaded datasets
loaded_datasets = {}

# Attempt to load a small portion of each dataset for sampling
for name in dataset_names:
    try:
        ds = load_dataset(name, split='train')  # Load only a small sample for speed
        loaded_datasets[name] = ds
    except Exception as e:
        loaded_datasets[name] = f"Failed to load: {e}"

# Show what was successfully loaded
loaded_datasets.keys()


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/675700 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

...

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/920 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/8134 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/926 [00:00<?, ? examples/s]

dict_keys(['fmars/wiki_stem', 'deepmind/aqua_rat', 'HuggingFaceTB/openstax_paragraphs', 'openlifescienceai/medmcqa', 'allenai/qasc'])

In [10]:
import openai
from tqdm import tqdm
import random
import pandas as pd
from datasets import load_dataset

# Set your API key
openai.api_key = "YOUR_OPENAI_API_KEY"
TARGET_TOKENS = 51_200_000
AVG_TOKENS_PER_CHUNK = 512
MAX_CHUNKS = TARGET_TOKENS // AVG_TOKENS_PER_CHUNK


# Final storage
final_rows = []

# Prompt template
def build_prompt(original_text):
    return f"""Rewrite the following passage by keeping its core meaning but changing the explicit details (like numbers, names, scenarios) to plausible alternatives:{original_text}"""

# Sample and call API
for name, dataset in tqdm(loaded_datasets.items()):
    if isinstance(dataset, str):  # Skip failed loads
        continue

    # Calculate approximate number of chunks per dataset (equal share)
    chunks_per_dataset = min(MAX_CHUNKS // len([d for d in loaded_datasets.values() if not isinstance(d, str)]), len(dataset))

    sampled_chunks_from_dataset = dataset.select(random.sample(range(len(dataset)), chunks_per_dataset))
    sampled_chunks.extend(sampled_chunks_from_dataset)

for sample in tqdm(sampled_chunks):
    original_text = sample.get("text") or sample.get("question") or sample.get("content")
    prompt = build_prompt(original_text)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful data augmentation assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=256
    )
    new_text = response['choices'][0]['message']['content'].strip()
    final_rows.append({"text": new_text, "source": "unknown_source"})


# Convert to DataFrame
augmented_df = pd.DataFrame(final_rows)

100%|██████████| 5/5 [00:00<00:00, 19710.08it/s]
0it [00:00, ?it/s]


In [8]:
augmented_df.to_csv("augmented_dataset.csv", index=False)
